In [1]:
# Notebook for more data investigation
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.min_rows', 50)


import matplotlib.pyplot as plt
%matplotlib widget
plt.rcParams['figure.figsize'] = (12,8)
# plt.rcParams["image.cmap"] = "Dark2"
# plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.Dark2.colors)
plt.rcParams["image.cmap"] = "tab10"
plt.rcParams['axes.prop_cycle'] = plt.cycler(color=plt.cm.tab10.colors)
plt.rcParams['lines.markersize'] = 10
plt.rcParams['lines.linewidth'] = 3
%matplotlib widget
# plt.style.use('ggplot')
import matplotlib.colors as colors
# cmap = plt.cm.get_cmap('Dark2',len(ageGroups))
from matplotlib import cm # Colormaps

import locale
import matplotlib.dates as mdates
locale.setlocale(locale.LC_TIME,"Danish")
# ax = plt.gca()
# ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b\n%Y'))
# # ax.xaxis.set_major_formatter(mdates.DateFormatter('%Y \n %B'))

import os
# import csv
import math


from datetime import date


saveFigures = True
print('saveFigures is set to: '+str(saveFigures))

print('Done loading packages')

def rnMean(data,meanWidth):
    return np.convolve(data, np.ones(meanWidth)/meanWidth, mode='valid')
def rnTime(t,meanWidth):
    return t[math.floor(meanWidth/2):-math.ceil(meanWidth/2)+1]


saveFigures is set to: True
Done loading packages


In [2]:
ssidatapath = "ssi_data"
rootdir = os.getcwd() +"/" + ssidatapath


for subdir, dirs, files in os.walk(rootdir):
    if not len(files) == 0:
        latestdir = subdir
        latestDate = pd.to_datetime(subdir[-10:])

print(latestdir)
print(latestDate)

# latestdir = rootdir+ '/SSI_data_2021-04-29'
# latestDate = pd.to_datetime(latestdir[-10:])

# print(latestdir)
# print(latestDate)

c:\Users\rakrpe\Documents\PandemiX\GithubRepos\PandemiX\DanskeData/ssi_data\SSI_data_2021-05-11
2021-05-11 00:00:00


In [3]:
# Load the data
fulldfPCR = pd.read_csv(latestdir+'/Test_pos_over_time.csv',delimiter = ';',dtype=str)
fulldfAG = pd.read_csv(latestdir+'/Test_pos_over_time_antigen.csv',delimiter = ';',dtype=str)

# # Cut out the last two rows of summary numbers
dfPCR = fulldfPCR.iloc[:-2,:].copy()
dfAG = fulldfAG.iloc[:-2,:].copy()

In [4]:
dfPCR["PosPct"] = pd.to_numeric(dfPCR["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfPCR["Date"] = dfPCR["Date"].astype('datetime64[D]')
dfAG["PosPct"] = pd.to_numeric(dfAG["PosPct"].astype(str).apply(lambda x: x.replace(',','.')))
dfAG["Date"] = dfAG["Date"].astype('datetime64[D]')

rows_to_fix_period_in = ["NewPositive","NotPrevPos","PrevPos","Tested","Tested_kumulativ"]
for name in rows_to_fix_period_in:
    dfAG[name] = pd.to_numeric(dfAG[name].astype(str).apply(lambda x: x.replace('.','')))
    dfPCR[name] = pd.to_numeric(dfPCR[name].astype(str).apply(lambda x: x.replace('.','')))


In [5]:
# Load the data

dfAG2 = pd.read_csv(latestdir+'/Antigentests_pr_dag.csv',delimiter = ';')

dfAG2["Dato"] = dfAG2["Dato"].astype('datetime64[D]')

In [6]:
# Cut off early data in pcr dataframe
dfPCR = dfPCR[dfPCR.Date >= dfAG.iloc[0].Date]

In [7]:
# dfPCR.tail()
# dfAG
firstDate = np.datetime64('2021-01-01')
dfPCR = dfPCR[dfPCR.Date >= firstDate]
dfAG = dfAG[dfAG.Date >= firstDate]

In [8]:
dfPCR = dfPCR.iloc[:-1]
dfAG = dfAG.iloc[:-1]

In [9]:
dfPCR.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
464,2021-05-05,1059,190251,0.6,6602,196853,28039948
465,2021-05-06,881,226454,0.4,8390,234844,28274792
466,2021-05-07,900,193759,0.5,6522,200281,28475073
467,2021-05-08,847,140622,0.6,4310,144932,28620005
468,2021-05-09,795,156852,0.5,4621,161473,28781478


In [10]:
curDates = dfPCR.Date

fig,ax1 = plt.subplots(1,1)

PCRpos = dfPCR.NewPositive
AGpos = dfAG.NewPositive
ax1.plot(curDates,PCRpos)
ax1.plot(curDates,AGpos)

plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [11]:
fig,ax1 = plt.subplots(1,1)

# ax1.plot(dfAG2.Dato,dfAG2.AG_testede) 
ax1.plot(dfAG2.Dato,np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos),'.-')
ax1.plot(rnTime(dfAG2.Dato,7),rnMean(np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos),7),'--')

truePosRate = np.divide(dfAG2.AGpos_PCRpos,dfAG2.AG_pos)
avgTruePosRate = np.mean(truePosRate[-30:])

ax1.plot(dfAG2.Dato,avgTruePosRate * np.ones(dfAG2.Dato.shape),'k:')


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [12]:

fig,ax1 = plt.subplots(1,1)

# PCRpos = dfPCR.NewPositive
# AGpos = dfAG.NewPositive
# ax1.plot(curDates,PCRpos)
ax1.plot(curDates,AGpos,label='AG+')
ax1.plot(curDates,AGpos*avgTruePosRate,label='AG+, times true-positive rate')
ax1.plot(dfAG2.Dato,dfAG2['AGpos_PCRpos'],label='AG+, PCR+ confirmed')

ax1.legend()
ax1.grid()
ax1.set_ylim(bottom=0)
ax1.set_xlim(left=np.datetime64('2021-03-01'))
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [13]:
dfPCR.tail()

,Date,NewPositive,NotPrevPos,PosPct,PrevPos,Tested,Tested_kumulativ
464,2021-05-05,1059,190251,0.6,6602,196853,28039948
465,2021-05-06,881,226454,0.4,8390,234844,28274792
466,2021-05-07,900,193759,0.5,6522,200281,28475073
467,2021-05-08,847,140622,0.6,4310,144932,28620005
468,2021-05-09,795,156852,0.5,4621,161473,28781478


In [14]:

fig,ax1 = plt.subplots(1,1)

PCRpos_nonAG = PCRpos.values -(AGpos.values*avgTruePosRate)

normFactor = (np.divide(100000,dfPCR['Tested']))**0.7

PCRpos_nonAG_corr = PCRpos_nonAG*normFactor

ax1.plot(curDates,PCRpos,'b.-',linewidth=0.5,markersize=2)
ax1.plot(curDates,PCRpos_nonAG,'m.-',linewidth=0.5,markersize=2)
ax1.plot(curDates,PCRpos_nonAG_corr,'g.-',linewidth=0.5,markersize=2)
meanWidth = 7
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos,meanWidth),'b',label='PCR positiv')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG,meanWidth),'m',label='PCR positiv, uden AG konf.')
ax1.plot(rnTime(curDates,meanWidth),rnMean(PCRpos_nonAG_corr,meanWidth),'g',label='PCR positiv, uden AG konf., korrigeret')
# ax1.plot(curDates,AGpos,label='AG+')
# ax1.plot(curDates,AGpos*avgTruePosRate,label='AG+, times true-positive rate')
# ax1.plot(dfAG2.Dato,dfAG2['AGpos_PCRpos'],label='AG+, PCR+ confirmed')

curFont = 18
ax1.legend(fontsize=curFont)
ax1.grid()
ax1.set_ylim(bottom=0,top=1200)
ax1.set_xlim(left=np.datetime64('2021-03-01'))

ax1.xaxis.set_major_formatter(mdates.DateFormatter('%d\n%b'))

# plt.xticks(fontsize=curFont,rotation=90)
plt.xticks(fontsize=curFont)
plt.yticks(fontsize=curFont)
ax1.set_ylabel('Antal',fontsize=curFont)
plt.tight_layout()

if saveFigures:
    plt.savefig('figs/PCRKorrigering')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …